In [1]:
from tools.project import INPUT_PATH, LOGS_PATH, OUTPUT_PATH, MODELS_PATH, RAW_PATH
import torch
import os
from datasets import Audio, load_dataset, Dataset
from src.jamendo_utils import read_file
from audiocraft.utils.notebook import display_audio
import numpy as np
import pandas as pd
from transformers import ClapConfig, ClapModel, AutoFeatureExtractor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import default_collate
import tqdm
import json
from audiocraft.data.audio import audio_read, audio_write
from audiocraft.data.audio_utils import convert_audio_channels, convert_audio


import umap
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from collections import Counter
from audiocraft.models import MusicGen

import random

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
# df.to_json(RAW_PATH('ds-indexes', 'song-describer.jsonl'), lines=True, indent=4, orient="records", force_ascii=False)
df = pd.read_json(RAW_PATH('ds-indexes', 'song-describer.jsonl'), lines=True)
for name in ['genre', 'instrument', 'mood/theme', 'text_clst']:
    df[name] = df[name].apply(lambda x: set(x))
df

,track_id,clst,genre,instrument,mood/theme,path,descriptions
0,1004034,9,"{progressive, trance, electronic, house}",{},{},34/1004034.2min.mp3,[Electronic music that has a constant melody t...
1,1007274,1,{easylistening},{},{},74/1007274.2min.mp3,[acoustic guitar solo track with consistent rh...
2,1009671,3,"{pop, electronic, rock, popfolk}","{drum, piano, synthesizer, acousticguitar}","{entertainment, commercial}",71/1009671.2min.mp3,"[A rock song with a country vibe, it has male ..."
3,1009672,3,"{pop, electronic}","{drum, piano, synthesizer, bass}","{entertainment, fashion, commercial}",72/1009672.2min.mp3,[A joyful and lively song that will make you w...
4,1009673,5,"{pop, folk, popfolk}","{drum, piano, synthesizer, acousticguitar}","{communication, entertainment, commercial}",73/1009673.2min.mp3,[Upbeat song that has a humming riff accompani...
...,...,...,...,...,...,...,...
701,976784,9,"{lounge, jazz, electronic}","{violin, synthesizer, electricpiano}",{relaxing},84/976784.2min.mp3,[upbeat electronic music with occasional melod...
702,976785,9,"{lounge, jazz, electronic}","{violin, synthesizer, electricpiano}",{relaxing},85/976785.2min.mp3,[Techno-house track that lopes and has synthet...
703,981782,5,"{pop, folk, popfolk}",{bells},"{happy, christmas}",82/981782.2min.mp3,[Sweet French song performed by a soft male vo...
704,986583,5,"{chanson, folk, popfolk}",{},{},83/986583.2min.mp3,[A poetic song by a French male singer-songwri...


In [19]:
df[df['mood/theme'].apply(lambda x: x.issuperset(['happy']))]

,track_id,clst,genre,instrument,mood/theme,path
27,1051207,8,"{singersongwriter, folk, popfolk}","{guitar, acousticguitar}","{happy, hopeful}",07/1051207.2min.mp3
39,1062831,9,"{soundtrack, pop, electronic}",{},"{happy, motivational}",31/1062831.2min.mp3
64,1107251,6,{hiphop},"{piano, electricguitar}","{happy, summer, relaxing}",51/1107251.2min.mp3
86,1134446,0,{electronic},"{synthesizer, keyboard}",{happy},46/1134446.2min.mp3
94,1143012,1,{popfolk},{},"{communication, motivational, advertising, cor...",12/1143012.2min.mp3
95,1143013,5,"{pop, easylistening}",{},"{motivational, positive, advertising, corporat...",13/1143013.2min.mp3
100,1153922,2,{pop},{ukulele},"{fun, motivational, positive, entertainment, c...",22/1153922.2min.mp3
120,1162697,0,{easylistening},{},"{zen, corporate, advertising, joyful, happy, o...",97/1162697.2min.mp3
171,1209452,11,{pop},{},"{communication, motivational, advertising, cor...",52/1209452.2min.mp3
172,1210700,3,"{rock, pop, folk}",{},"{cool, fun, motivational, groovy, melodic, joy...",00/1210700.2min.mp3
